<font size="5">**Requirements to run this project**</font>
* <font size="3">Python 2 - 3.7 </font>
* <dd><font size="3">Tensorflow 1.3 - 1.15 </font></dd>
* <dd><font size="3">MySQL</font></dd>
* <dd><font size="3">sklearn, numpy </font></dd>
* <dd><font size="3">Pyro4 </font></dd>

Check Tensorflow version, it should be (1.15) in order to work properly with the slim DNN models



In [1]:
import tensorflow as tf
print(tf.__version__)

1.15.0


<font size="5">**Importing libraries and python scripts**</font>
<br /><br />
<font size="3">Before starting, carefully go through the Setup in the README.md to obtain necessary images and checkpoints.  
</font>

In [2]:
#!pip install mysql-connector-python
#!pip install --user mysql-python
#!pip install Pyro4
#!pip install --upgrade pip
#!pip install --user tensorflow==1.15
import mysql
import mysql.connector as mysql
from nets import nets_factory
from preprocessing import preprocessing_factory
import model
import analysis
import os
import inference
import image_generator
import numpy as np

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



<font size="3"> If some libraries are not installed, install them using pip </font>

<font size="5">**Starting the image generator class**</font>
<br /><br />
<font size="3">This code starts the image generator class and can be used to check if the <u>images</u> and <u>dataset files</u> are stored properly and are accessible.</font>

In [3]:
image_generator.main()
imgGen = image_generator.ImageGenerator('images/val/images', 'images/val/val.txt', 'images/val/synset_words.txt')
label_list = imgGen.get_label_list()      # this is the correct label list that will be used to evaluate DNN performance

images/val/images\ILSVRC2012_val_00013333.JPEG dugong, Dugong dugon
images/val/images\ILSVRC2012_val_00031130.JPEG electric fan, blower
images/val/images\ILSVRC2012_val_00045117.JPEG typewriter keyboard
images/val/images\ILSVRC2012_val_00017938.JPEG pug, pug-dog
images/val/images\ILSVRC2012_val_00007441.JPEG home theater, home theatre
images/val/images\ILSVRC2012_val_00017828.JPEG file, file cabinet, filing cabinet
images/val/images\ILSVRC2012_val_00015537.JPEG pool table, billiard table, snooker table
images/val/images\ILSVRC2012_val_00029990.JPEG hotdog, hot dog, red hot
images/val/images\ILSVRC2012_val_00010334.JPEG passenger car, coach, carriage
images/val/images\ILSVRC2012_val_00030915.JPEG American Staffordshire terrier, Staffordshire terrier, American pit bull terrier, pit bull terrier


<font size="5">**Testing inference on one image**</font>
<br /><br />
<font size="3">This code initializes the models and infers the first image with each of them. It is used to check if <u>model checkpoints</u> are accessible.</font>

In [4]:
model_name = 'inception_v4'
model = inference.DNN_Model('tf', model_name, 'model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt')
results = model.time_model(['images/val/images/ILSVRC2012_val_00000001.JPEG'], 1, [])
print(results)

model_name = 'resnet_v1_152'
model = inference.DNN_Model('tf', model_name, 'model_data/tensorflow/checkpoints/resnet_v1_152/resnet_v1_152.ckpt')
results = model.time_model(['images/val/images/ILSVRC2012_val_00000001.JPEG'], 1, [])
print(results)

model_name = 'mobilenet_v1'
model = inference.DNN_Model('tf', model_name, 'model_data/tensorflow/checkpoints/mobilenet_v1_1.0_224/mobilenet_v1_1.0_224.ckpt')
results = model.time_model(['images/val/images/ILSVRC2012_val_00000001.JPEG'], 1, [])
print(results)

Skipping first inference...

Preprocessing image: 1 of 1


INFO:tensorflow:Scale of 0 disables regularizer.

Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.flatten instead.

INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt
Image: 1 of 1	Average Time: 1078.0
Done!

Preprocessing image: 1 of 1
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt
Image: 1 of 1	Average Time: 858.0
[([858.0], [[0.9475860595703125, 0.011890850029885769, 0.0072621447034180164, 0.0031914273276925087, 0.003110471647232771, 0.002808748045936227, 0.0014050431782379746, 0.0009503407636657357, 0.0008339508203789592, 0.0006610105047002435, 0.0006372712086886168, 0.0005837938515469432, 0.0005329005070962012, 0.00038708027568645775, 0.0003691568272188306, 0.0003241834929212928, 0.0001592216431

<font size="5">**Conecting to MySQL**</font>
<br /><br />
<font size="3">This code can be used to check if connection to MySQL can be established. You should insert your own <font color = "green">hostname</font>, <font color = "green">username</font>, and <font color = "green">password</font>. These values should also be used in *connect_db* method in *database.py* file (change it).</font>

In [5]:
db = mysql.connect(
    host = "127.0.0.1", #could be "localhost"
    user = "root",
    passwd = "Vj55577022077555."
)

<font size="5">**Obtaining inference results**</font>
<br /><br />
<font size="3">This code retrieves checkpoints for the models and obtains <font color = "green">inference results</font> on selected images.</font>


In [7]:
model_i_name = 'inception_v4'
model_i = inference.DNN_Model('tf', model_i_name, 'model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt')

model_r_name = 'resnet_v1_152'
model_r = inference.DNN_Model('tf', model_r_name, 'model_data/tensorflow/checkpoints/resnet_v1_152/resnet_v1_152.ckpt')

model_m_name = 'mobilenet_v1'
model_m = inference.DNN_Model('tf', model_m_name, 'model_data/tensorflow/checkpoints/mobilenet_v1_1.0_224/mobilenet_v1_1.0_224.ckpt')

models = [model_i, model_r, model_m]
model_names = [model_i_name, model_r_name, model_m_name]
results = []

img_nums = []
img_path_list = []
img_labels = []
for i in range(1, 201):                           # here, the range of images can be selected
    img_nums.append(i)
for i in range(len(img_nums)):
    img_path, img_label = imgGen.get_image_data(img_nums[i])
    img_path_list.append(img_path)
    img_labels.append(img_label)
for i in range(len(models)):
    results.append(models[i].time_model(img_path_list, 1, []))                # 0 - inception, 1 - resnet, 2 - mobilenet

Skipping first inference...
Preprocessing image: 1 of 1
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt
Image: 1 of 1	Average Time: 742.0
Done!

Preprocessing image: 1 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt
Image: 1 of 200	Average Time: 958.0
Preprocessing image: 2 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt
Image: 2 of 200	Average Time: 1004.0
Preprocessing image: 3 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt
Image: 3 of 200	Average Time: 1068.0
Preprocessing image: 4 of 200
INFO:tensorflow:Scale of 0 disables regularizer.


Image: 36 of 200	Average Time: 705.0
Preprocessing image: 37 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt
Image: 37 of 200	Average Time: 694.0
Preprocessing image: 38 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt
Image: 38 of 200	Average Time: 728.0
Preprocessing image: 39 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt
Image: 39 of 200	Average Time: 694.0
Preprocessing image: 40 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt
Image: 40 of 200	Average Time: 747.0
Preprocessing image: 41 of 200
INFO:tensorflow:Scale of 0 disables 

Image: 73 of 200	Average Time: 736.0
Preprocessing image: 74 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt
Image: 74 of 200	Average Time: 832.0
Preprocessing image: 75 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt
Image: 75 of 200	Average Time: 810.0
Preprocessing image: 76 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt
Image: 76 of 200	Average Time: 1169.0
Preprocessing image: 77 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt
Image: 77 of 200	Average Time: 786.0
Preprocessing image: 78 of 200
INFO:tensorflow:Scale of 0 disables

INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt
Image: 110 of 200	Average Time: 1032.0
Preprocessing image: 111 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt
Image: 111 of 200	Average Time: 1205.0
Preprocessing image: 112 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt
Image: 112 of 200	Average Time: 1165.0
Preprocessing image: 113 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt
Image: 113 of 200	Average Time: 1144.0
Preprocessing image: 114 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/inception_v4/inc

Image: 146 of 200	Average Time: 1040.0
Preprocessing image: 147 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt
Image: 147 of 200	Average Time: 1007.0
Preprocessing image: 148 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt
Image: 148 of 200	Average Time: 990.0
Preprocessing image: 149 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt
Image: 149 of 200	Average Time: 1398.0
Preprocessing image: 150 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/inception_v4/inception_v4.ckpt
Image: 150 of 200	Average Time: 1169.0
Preprocessing image: 151 of 200
INFO:tensorflow:Scale 

...

Image: 32 of 200	Average Time: 159.0
Preprocessing image: 33 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/mobilenet_v1_1.0_224/mobilenet_v1_1.0_224.ckpt
Image: 33 of 200	Average Time: 154.0
Preprocessing image: 34 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/mobilenet_v1_1.0_224/mobilenet_v1_1.0_224.ckpt
Image: 34 of 200	Average Time: 153.0
Preprocessing image: 35 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/mobilenet_v1_1.0_224/mobilenet_v1_1.0_224.ckpt
Image: 35 of 200	Average Time: 168.0
Preprocessing image: 36 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/mobilenet_v1_1.0_224/mobilenet_v1_1.0_224.ckpt
Image: 36 of 200	Average Time: 156.0
Pre

INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/mobilenet_v1_1.0_224/mobilenet_v1_1.0_224.ckpt
Image: 67 of 200	Average Time: 350.0
Preprocessing image: 68 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/mobilenet_v1_1.0_224/mobilenet_v1_1.0_224.ckpt
Image: 68 of 200	Average Time: 221.0
Preprocessing image: 69 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/mobilenet_v1_1.0_224/mobilenet_v1_1.0_224.ckpt
Image: 69 of 200	Average Time: 170.0
Preprocessing image: 70 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/mobilenet_v1_1.0_224/mobilenet_v1_1.0_224.ckpt
Image: 70 of 200	Average Time: 170.0
Preprocessing image: 71 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters fro

Image: 101 of 200	Average Time: 168.0
Preprocessing image: 102 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/mobilenet_v1_1.0_224/mobilenet_v1_1.0_224.ckpt
Image: 102 of 200	Average Time: 163.0
Preprocessing image: 103 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/mobilenet_v1_1.0_224/mobilenet_v1_1.0_224.ckpt
Image: 103 of 200	Average Time: 160.0
Preprocessing image: 104 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/mobilenet_v1_1.0_224/mobilenet_v1_1.0_224.ckpt
Image: 104 of 200	Average Time: 162.0
Preprocessing image: 105 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/mobilenet_v1_1.0_224/mobilenet_v1_1.0_224.ckpt
Image: 105 of 200	Average Time: 

Image: 135 of 200	Average Time: 230.0
Preprocessing image: 136 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/mobilenet_v1_1.0_224/mobilenet_v1_1.0_224.ckpt
Image: 136 of 200	Average Time: 158.0
Preprocessing image: 137 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/mobilenet_v1_1.0_224/mobilenet_v1_1.0_224.ckpt
Image: 137 of 200	Average Time: 224.0
Preprocessing image: 138 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/mobilenet_v1_1.0_224/mobilenet_v1_1.0_224.ckpt
Image: 138 of 200	Average Time: 354.0
Preprocessing image: 139 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/mobilenet_v1_1.0_224/mobilenet_v1_1.0_224.ckpt
Image: 139 of 200	Average Time: 

Image: 169 of 200	Average Time: 157.0
Preprocessing image: 170 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/mobilenet_v1_1.0_224/mobilenet_v1_1.0_224.ckpt
Image: 170 of 200	Average Time: 158.0
Preprocessing image: 171 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/mobilenet_v1_1.0_224/mobilenet_v1_1.0_224.ckpt
Image: 171 of 200	Average Time: 159.0
Preprocessing image: 172 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/mobilenet_v1_1.0_224/mobilenet_v1_1.0_224.ckpt
Image: 172 of 200	Average Time: 167.0
Preprocessing image: 173 of 200
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from model_data/tensorflow/checkpoints/mobilenet_v1_1.0_224/mobilenet_v1_1.0_224.ckpt
Image: 173 of 200	Average Time: 

<font size="3">In these cells, results are processed to obtain <font color = "green">top-1</font> and <font color = "green">top-5 satisfaction (0 or 1)</font> for each image by each of the DNNs, as well as <font color = "green">inference times</font>.

In [8]:
top_1 = [[], [], []]
top_5 = [[], [], []]
performance = [[], [], []]

In [14]:
for j in range(len(models)):
    for i in range(len(img_nums)):
        performance[j].append(results[j][i][0][0])
        top_1[j].append(int(analysis.top_n_correct(np.array(results[j][i][1][0]), np.array(results[j][i][2][0]), img_labels[i], label_list, 1)))
        top_5[j].append(int(analysis.top_n_correct(np.array(results[j][i][1][0]), np.array(results[j][i][2][0]), img_labels[i], label_list, 5)))

<font size="5">**Creating a database**</font>
<br /><br />
<font size="3">Here, we create the database that will hold <font color = "green">image names</font> in the *images* table and <font color = "green">inference results</font> in the *exec_data* table.</font>


In [18]:
#Change host, user, passwd for your own

db = mysql.connect(
    host = "127.0.0.1",
    user = "root",
    passwd = "Vj55577022077555."
)

cursor = db.cursor()

cursor.execute("CREATE DATABASE IF NOT EXISTS inference")

db = mysql.connect(
    host = "127.0.0.1",
    user = "root",
    passwd = "Vj55577022077555.",
    database = "inference"
)

cursor = db.cursor()

<font size="3">Inserting the image data</font>

In [19]:
cursor.execute("CREATE TABLE IF NOT EXISTS images (img_num int(5) NOT NULL, filename varchar(35) NOT NULL, KEY img_num (img_num)) ENGINE=InnoDB;")

filename = []

for i in range(len(img_nums)):
    filename.append(imgGen.get_image_filename(img_nums[i]))
for i in range(len(img_nums)):
    query = "INSERT INTO images (img_num, filename) VALUES (%s, %s)"
    values = (img_nums[i], filename[i])
    cursor.execute(query, values)

db.commit()

<font size="3">Inserting the inference results</font>

In [20]:
cursor.execute("CREATE TABLE IF NOT EXISTS exec_data (img_num int(5) NOT NULL, model_name varchar(14) NOT NULL, top_1 int(1) NOT NULL, top_5 int(1) NOT NULL, performance int(7) NOT NULL, KEY img_num (img_num)) ENGINE=InnoDB;")

for i in range(len(img_nums)):
    for j in range(len(models)):
        query = "INSERT INTO exec_data (img_num, model_name, top_1, top_5, performance) VALUES (%s, %s, %s, %s, %s)"
        values = (img_nums[i], model_names[j], top_1[j][i], top_5[j][i], performance[j][i])
        cursor.execute(query, values)

db.commit()

<font size="5">**Obtaining information from the database**</font>
<br /><br />
<font size="3">We use *database.py* to obtain the <font color="green">best top-1</font> and <font color="green">top-5</font> models for each image.</font>

In [21]:
import database as db

In [22]:
myDB = db.connect_db("inference")
list_model = ["mobilenet_v1", "inception_v4", "resnet_v1_152"]

for i in range(len(img_nums)):
    print('Best top-1 model for img num \''+str(img_nums[i])+'\' is ')
    print(db.determine_best_top_n_model("inference", img_nums[i], list_model, 1))
    print('\n' * 1)
    print('Best top-5 model for img num =\''+str(img_nums[i])+'\' is ')
    print(db.determine_best_top_n_model("inference", img_nums[i], list_model, 5))
    print('\n' * 2)

Best top-1 model for img num '1' is 
mobilenet_v1


Best top-5 model for img num ='1' is 
mobilenet_v1



Best top-1 model for img num '2' is 
failed


Best top-5 model for img num ='2' is 
mobilenet_v1



Best top-1 model for img num '3' is 
mobilenet_v1


Best top-5 model for img num ='3' is 
mobilenet_v1



Best top-1 model for img num '4' is 
mobilenet_v1


Best top-5 model for img num ='4' is 
mobilenet_v1



Best top-1 model for img num '5' is 
inception_v4


Best top-5 model for img num ='5' is 
mobilenet_v1



Best top-1 model for img num '6' is 
inception_v4


Best top-5 model for img num ='6' is 
inception_v4



Best top-1 model for img num '7' is 
mobilenet_v1


Best top-5 model for img num ='7' is 
mobilenet_v1



Best top-1 model for img num '8' is 
failed


Best top-5 model for img num ='8' is 
inception_v4



Best top-1 model for img num '9' is 
mobilenet_v1


Best top-5 model for img num ='9' is 
mobilenet_v1



Best top-1 model for img num '10' is 
mobilenet_v1


Best 

mobilenet_v1



Best top-1 model for img num '83' is 
mobilenet_v1


Best top-5 model for img num ='83' is 
mobilenet_v1



Best top-1 model for img num '84' is 
resnet_v1_152


Best top-5 model for img num ='84' is 
mobilenet_v1



Best top-1 model for img num '85' is 
resnet_v1_152


Best top-5 model for img num ='85' is 
inception_v4



Best top-1 model for img num '86' is 
mobilenet_v1


Best top-5 model for img num ='86' is 
mobilenet_v1



Best top-1 model for img num '87' is 
failed


Best top-5 model for img num ='87' is 
inception_v4



Best top-1 model for img num '88' is 
mobilenet_v1


Best top-5 model for img num ='88' is 
mobilenet_v1



Best top-1 model for img num '89' is 
mobilenet_v1


Best top-5 model for img num ='89' is 
mobilenet_v1



Best top-1 model for img num '90' is 
mobilenet_v1


Best top-5 model for img num ='90' is 
mobilenet_v1



Best top-1 model for img num '91' is 
mobilenet_v1


Best top-5 model for img num ='91' is 
mobilenet_v1



Best top-1 model

resnet_v1_152



Best top-1 model for img num '163' is 
mobilenet_v1


Best top-5 model for img num ='163' is 
mobilenet_v1



Best top-1 model for img num '164' is 
mobilenet_v1


Best top-5 model for img num ='164' is 
mobilenet_v1



Best top-1 model for img num '165' is 
mobilenet_v1


Best top-5 model for img num ='165' is 
mobilenet_v1



Best top-1 model for img num '166' is 
mobilenet_v1


Best top-5 model for img num ='166' is 
mobilenet_v1



Best top-1 model for img num '167' is 
mobilenet_v1


Best top-5 model for img num ='167' is 
mobilenet_v1



Best top-1 model for img num '168' is 
mobilenet_v1


Best top-5 model for img num ='168' is 
mobilenet_v1



Best top-1 model for img num '169' is 
mobilenet_v1


Best top-5 model for img num ='169' is 
mobilenet_v1



Best top-1 model for img num '170' is 
mobilenet_v1


Best top-5 model for img num ='170' is 
mobilenet_v1



Best top-1 model for img num '171' is 
mobilenet_v1


Best top-5 model for img num ='171' is 
mobilenet

<font size="3">This function prints out the contents of the *images* table of the database.</font>

In [23]:
for i in range(len(filename)):
    print(db.get_img_num_database("inference", filename[i]))

(1, 'ILSVRC2012_val_00000001.JPEG')
(2, 'ILSVRC2012_val_00000002.JPEG')
(3, 'ILSVRC2012_val_00000003.JPEG')
(4, 'ILSVRC2012_val_00000004.JPEG')
(5, 'ILSVRC2012_val_00000005.JPEG')
(6, 'ILSVRC2012_val_00000006.JPEG')
(7, 'ILSVRC2012_val_00000007.JPEG')
(8, 'ILSVRC2012_val_00000008.JPEG')
(9, 'ILSVRC2012_val_00000009.JPEG')
(10, 'ILSVRC2012_val_00000010.JPEG')
(11, 'ILSVRC2012_val_00000011.JPEG')
(12, 'ILSVRC2012_val_00000012.JPEG')
(13, 'ILSVRC2012_val_00000013.JPEG')
(14, 'ILSVRC2012_val_00000014.JPEG')
(15, 'ILSVRC2012_val_00000015.JPEG')
(16, 'ILSVRC2012_val_00000016.JPEG')
(17, 'ILSVRC2012_val_00000017.JPEG')
(18, 'ILSVRC2012_val_00000018.JPEG')
(19, 'ILSVRC2012_val_00000019.JPEG')
(20, 'ILSVRC2012_val_00000020.JPEG')
(21, 'ILSVRC2012_val_00000021.JPEG')
(22, 'ILSVRC2012_val_00000022.JPEG')
(23, 'ILSVRC2012_val_00000023.JPEG')
(24, 'ILSVRC2012_val_00000024.JPEG')
(25, 'ILSVRC2012_val_00000025.JPEG')
(26, 'ILSVRC2012_val_00000026.JPEG')
(27, 'ILSVRC2012_val_00000027.JPEG')
(28, 'ILSV